In [1]:
import pandas as pd
import numpy as np
import json
import openpyxl

### choose sheet name "EN" or anything else for ISSUE

In [224]:
df=pd.read_excel(r"C:\Users\Palash Ashok Bhosale\Jupy\Projects\cbrelease\faq-assistant\data\faq.xlsx",sheet_name="ISEN")
# sheet="HI"

In [225]:
lang="EN"
CAT="IS"

## Recommendation map 

### Before getting priority df

In [226]:
def recommendation_qid_json_output(data, lang, file):
    

    global faq_bot
    
    faq_bot = []                                     # Initiating Json Empty list
    #############################################
    def seq2():                                      # L2 level Ques ids starting from 101
        num = 101
        while True:
            yield num
            num += 1

    gen2 = seq2() 
    
    unique_cat=data["Category"].unique()             # unique categories
    
    d={}                                             # categories: catID
    num=np.arange(0, len(unique_cat))

    for i in range(len(unique_cat)):
        d[unique_cat[i]]= f"{CAT}{lang}C{next(gen2)}"
        
    #############################################
    
    qa1={}                                           # dictionary for capturing ques and qid of lvl1 questions
    qa2={}                                           # dictionary for capturing ques and qid of lvl2 questions
    
    #############################################
    
    categ=[]
        
    count=0
        
    for cat in (data["Category"].unique()):
        
        #############################################

        

        def seq():                                       # L1 level Ques ids starting from 1001
            num = 101
            while True:
                yield int(num)
                num += 1

        gen = seq()

        #############################################

        def seq1():                                      # L2 level Ques ids starting from 101
            num = 101
            while True:
                yield num
                num += 1

        gen1 = seq1()                                   

        #############################################
        def reset_seq2(generator):
            generator.gi_frame.f_locals['num'] = 101 
        
        len_l1_que=len(data[data["Category"]==cat])
    

        for _, row in data[data["Category"]==cat].iterrows():
            cat_id = d[cat]
            
            category_type = row['Category Type']

            

            
            l1_que= row['Questions (Level 1)']
            
            l1_ans= row['Answer (Level 1)']
            l2_que= row['Questions (Level 2)']
            l2_ans= row['Answer (Level 2)']
            
            if row["Questions (Level 2)"] is np.nan:
                l2_que= np.nan
                l2_ans= np.nan
            else:
                l2_que= row['Questions (Level 2)']
                l2_ans= row['Answer (Level 2)']

        
            
            if l1_que in qa1.keys() and l2_que not in qa2.keys():
                if l2_que is not np.nan:
                    ques_id= qa1[l1_que]
                    ques_id1= f"{ques_id}L2Q{next(gen1)}"
#                 reset_seq2(gen1)
                else:
                    ques_id= qa1[l1_que]
                    ques_id1= np.nan
                
            else:
                ques_id= f"{cat_id}L1Q{next(gen)}"
                
                if l2_que is not np.nan:
                    
                    ques_id1= f"{ques_id}L2Q{next(gen1)}"
                    
                else:
                    ques_id1= np.nan



            faq_entry = {
                "catId": cat_id,
                "categoryType": category_type,
                "priority": 1,
                "recommendedQues": []
            }
            
            

                    

            if cat not in categ:
                for i in range(1):
                    ques_entry = {
                        "priority": 1,
                        "quesID": ques_id,
                        "recommendedQues": []
                    }
                    if l2_que is not np.nan:
                        

                    
                        recommended_ques_entry = {
                            "priority": 1,
                            "quesID": ques_id1
                        }
                        ques_entry["recommendedQues"].append(recommended_ques_entry)

                    faq_entry["recommendedQues"].append(ques_entry)



                faq_bot.append(faq_entry)
                categ.append(cat)


            else :
                if l1_que not in qa1.keys() and l2_que not in qa2.keys() and l2_que is not np.nan:

                    ques_entry = {
                        "priority": 1,
                        "quesID": ques_id,
                        "recommendedQues": []
                    }


#                     recommended_ques_entry = {
#                         "priority": 1,
#                         "quesID": ques_id1
#                     }
#                     ques_entry["recommendedQues"].append(recommended_ques_entry)

                    faq_bot[count]["recommendedQues"].append(ques_entry)
#                     print()
#                     print("you are in 2")
#                     print(ques_id1, ques_id)
                    pass

                elif l1_que not in qa1.keys() and l2_que is np.nan:

                    ques_entry = {
                        "priority": 1,
                        "quesID": ques_id,
                        "recommendedQues": []
                    }


                    faq_bot[count]["recommendedQues"].append(ques_entry)
#                     print()
#                     print("you are in 3")
#                     print(ques_id1, ques_id)
                    
                    pass
                for q in range(0, len_l1_que):
                    try:

                        if ques_id in faq_bot[count]["recommendedQues"][q]["quesID"] and l2_que is not np.nan:


                            
                            recommended_ques_entry = {
                                "priority": 1,
                                "quesID": ques_id1
                            }


                            faq_bot[count]["recommendedQues"][q]["recommendedQues"].append(recommended_ques_entry)
#                             print()
#                             print("you are in 1")
#                             print(ques_id1, ques_id)
                            

                    except:
#                         print()
#                         print("you are in else")
#                         print(ques_id1, ques_id)
                        pass
                

                
#                         reset_seq2(gen1)
#                         break
                
#                 break
#                 qa1[l1_que]=ques_id
#                 qa2[l2_que]=ques_id1
#                     print(faq_bot)
#                 break
            
#             print(qa2)
            qa1[l1_que]=ques_id
            qa2[l2_que]=ques_id1
#             break
        count+=1
            
                
#     print(qa2)
    data_outpu = {"faqBot": faq_bot}
    json_output = json.dumps(data_outpu, indent=2)
    with open(file, 'w') as json_file:
        json.dump(data_outpu, json_file, indent=2)

    return [qa1, qa2, d, faq_bot]

In [227]:
dict1=recommendation_qid_json_output(df, lang, f"{CAT}_recommendation_{lang}.json")

## que ans mapping for ISSUE {IS]

In [228]:
def qa_json_output(data, lang, file, lvl1_que, lvl2_que):
    qa1= lvl1_que
    qa2= lvl2_que
    ques_map = []
    lang_set = []
    l=[]
    
    for _, row in data.iterrows():

        cat=row['Category']
        category_type = row['Category Type']
#         priority = row['priority']
        
        
        l1_que= row['Questions (Level 1)']
        l2_que= row['Questions (Level 2)']
        l1_ans= row['Answer (Level 1)']
        l2_ans= row['Answer (Level 2)']


   
        
        for k, v in qa1.items():


            if k==l1_que and l.count(v)<1 :

                ques_map.append({
                  "quesId": v,
                  "ansVal": l1_ans,
                  "quesValue":k
                          

                })
                l.append(v)
            
                    
        for k, v in qa2.items():
#             print(k, l2_que)

            if k==l2_que and l.count(v)<1 :

                ques_map.append({
                  "quesId": v,
                  "ansVal": l2_ans,
                  "quesValue":k
                         
                })
                l.append(v)
            continue
            

            
    data_outpu={"quesMap": ques_map}
    with open(file, 'w') as json_file:
        json.dump(data_outpu, json_file, indent=2)
    return data_outpu

qa1=dict1[0]
qa2=dict1[1]
cat_dict=dict1[2]

qa_json_output(df, lang, f"{CAT}_qa_{lang}.json", qa1, qa2)

{'quesMap': [{'quesId': 'ISENC101L1Q101',
   'ansVal': "Please make sure you are using a government (official) Email ID for registration. If you are still not able to register, please get help from the 'Request for Help' link. If you still face the same issue, please connect with us between 9 AM to 5 PM on <teams_call_link> or email us at <email_configuration>",
   'quesValue': "I can't register, can you help?"},
  {'quesId': 'ISENC101L1Q101L2Q101',
   'ansVal': 'You may self-register yourself using your government (official) Email ID and choose your organization. Once approved, you will receive an Email to set your password. After setting the password, you will be able to login to the platform.',
   'quesValue': 'What should I do if my superior has not got me registered yet?'},
  {'quesId': 'ISENC101L1Q102',
   'ansVal': 'iGOT Karmayogi is for Indian Government Officials, therefore, you will need to use your Government (official) Email ID for registration. If you have any further ques

### Category map for ISSUE [IS]

In [229]:
def cat_json_output(data, d, lang, file):
    
       
    a=data["Category"].unique()# Category ids
    a=list(a)
    
    
    category_map=[]
    for k,v in d.items():
        
        category_map.append({
            "catId": v,
            "catName": k
        })

        
        
    output_cat = {"categoryMap": category_map}


    with open(file, 'w') as json_file:
        json.dump(output_cat, json_file, indent=2)
        
    return output_cat

cat_dict

cat_json_output(df, cat_dict, lang, f"{CAT}_cat_map_{lang}.json")

{'categoryMap': [{'catId': 'ISENC101', 'catName': 'Registration'},
  {'catId': 'ISENC102', 'catName': 'Login'},
  {'catId': 'ISENC103', 'catName': 'Profile'},
  {'catId': 'ISENC104', 'catName': 'Hubs'},
  {'catId': 'ISENC105', 'catName': 'Learning, Assessment and Certifications'},
  {'catId': 'ISENC106', 'catName': 'Other'}]}

In [204]:
#importing Libraries
import pandas as pd
import numpy as np
import json

# importing path of excel and Json files 

path_excel=r"C:\Users\Palash Ashok Bhosale\Jupy\Projects\FAQ - Copy\palash\priority_sort\Final_sheet_16_10.xlsx"
path_priority=r"C:\Users\Palash Ashok Bhosale\Jupy\Projects\FAQ - Copy\palash\priority_sort\priority_dff.xlsx"


# Creating class api
class api():
    
    # while calling class API, we must pass lang=Language("EN", "HI", etc) and 
    # p_cat=parent category("IN": information, "IS": issue)

    # while calling class API, we must pass lang=Language("EN", "HI", etc) and 
    # p_cat=parent category("IN": information, "IS": issue)

    def __init__(self, lang, p_cat):
        self.lang = lang
        self.p_cat = p_cat
        self.load_paths()
        self.load_dataframes()
        self.load_all_maps()
        ####################################################
        
    def load_paths(self):
        # Define file paths based on language and parent category
        base_path = r"C:\Users\Palash Ashok Bhosale\Jupy\Projects\FAQ - Copy\palash\priority_sort"
        self.path_recommend = f"{base_path}/{self.p_cat}_recommendation_{self.lang}.json"
        self.path_cat_map = f"{base_path}/{self.p_cat}_cat_map_{self.lang}.json"
        self.path_que_map = f"{base_path}/{self.p_cat}_qa_{self.lang}.json"
        self.path_excel = path_excel
        self.path_priority = path_priority
        ####################################################
        
    def load_dataframes(self):
        # as all the data is in on excel file we need to fetch specific sheet
        sheet_name=f"{self.p_cat}{self.lang}"
        self.df = pd.read_excel(self.path_excel, sheet_name=sheet_name)
        ####################################################
        
    def load_all_maps(self):
        with open(self.path_recommend, "r", encoding="utf-8") as rec_file:
            rec_data = json.load(rec_file)
            self.recommend1 = rec_data["faqBot"]

        with open(self.path_cat_map, "r", encoding="utf-8") as cat_file:
            cat_data = json.load(cat_file)
            self.cat_map = cat_data

        with open(self.path_que_map, "r", encoding="utf-8") as que_file:
            que_data = json.load(que_file)
            self.que_map = que_data
        ####################################################
        
    def generate_priorit_recommend(self, p_data=0): #method for giving priority for existing recommendation_map
        global data

        
        config=[]
        p=1
        data=self.df
        faqBot=self.recommend1     
        lang=self.lang
        ####################################################
        
        dict_container = {}                # Create a dictionary to hold other dictionaries
        dict_container[f"cat{p}_clicks"] = {}
        for i in range(1,len(data["Category"].unique())+1):#for loop to auto generate dictionaary for no. of categories 
            dict_name = f"cat{i}_L1_clicks" # Create the dynamic dictionary name
            dict_container[dict_name] = {}  # Create an empty dictionary and store it in the container
            dict_name = f"cat{i}_L2_clicks"  
            dict_container[dict_name] = {}  
#             print(dict_container)

        priority_rank={} #initiating dict to store ids and respective priority rank
        ##################################################

        #Iterating over existing recommendation map and setting priority based on priority rank dictionary 
        if type(p_data)!=int:# checking if priority dataframe is passed otherwise p_data is taken as 0 by default
            num=1 # num variable is used for incrementation
            
            for index,row in p_data.iterrows():#for loop for iterating over priority dataframe
                if index.startswith(f"{self.p_cat}{self.lang}C") and 0<len(index)<=8:
                    dict_container[f"cat{p}_clicks"][index]=row["clicks"]# creating cat clicks dict in parent dict 

                for i in range(1,len(data["Category"])):# creating L1 and L2 clicks dict in parent dict
                    if index.startswith(f"{self.p_cat}{self.lang}C10{i}") and 8<len(index)==14:
                        dict_container[f"cat{i}_L1_clicks"][index]=row['clicks']

                    elif index.startswith(f"{self.p_cat}{self.lang}C10{i}") and 14<len(index)==20:
                        dict_container[f"cat{i}_L2_clicks"][index]=row['clicks']
            num+=1
            ####################################################
            
            for k,v in dict_container.items():# for loop for reverse sorting of all dicts in parent dict
                dict_container[k]={k:v for k,v in (sorted(dict_container[k].items(), key=lambda x:x[1],reverse=True))}          
            count=1 #count variable is initiated to set priority rank for sorted ids from 1 to length of that dict

            for key,value in dict_container.items():# for loop for iterating over each dictionary in parent dict

                for k,v in dict_container[key].items():# for loop for iterating over key i.e dicts in parent dict 
                    priority_rank[k]=count                             # and value i.e dict having ids and clicks
                    count+=1
                count=1 
            count=1
           #################################################### 

            # grouping l1 level questions and l2 level questions based on ids
            # L1 and L2 ids are captured by iterating over existing recommendation map
            l=[]
            for cc in range(len(data["Category"].unique())):
                for i in range (len(faqBot[cc]["recommendedQues"])):
                    L1_id=faqBot[cc]["recommendedQues"][i]["quesID"] 
                    for j in range (len(faqBot[cc]["recommendedQues"][i]["recommendedQues"])):
                        L2_id=faqBot[cc]["recommendedQues"][i]["recommendedQues"][j]["quesID"]
                        l.extend([[L1_id, L2_id]])
            ####################################################            
            li={}# creating dictionary for capturing L1 ids of particular category and
                 #L2 ids of each different L1 qids as key and value is clicks on that ids 
                 #Fetching clicks of that id from priority dataframe passed in this function
            a=0
            for i in l:
                if l[a][0] not in li.keys():
                    try:
                        li[l[a][0]]= [[p_data.loc[l[a][1]][0], l[a][1]]] #if id is not in dict "li" then new key will be created
                    except:
                        print("no id found in telemetry for:" + str(i))

                else:
                    try:
                        li[l[a][0]].extend([[p_data.loc[l[a][1]][0], l[a][1]]]) #if id is in dict "li" then value will be added
                    except:
                        print("no id found in telemetry for:" + str(i))
                a+=1
            a=0
            ####################################################
            
            for k,v in li.items():
                num=1
                lll=sorted(v, reverse=True)

                for i in range(len(lll)):
                    priority_rank[lll[i][1]]=num  # populating priority based on number of clicks
                    num+=1
                num=1
            ####################################################
            
            #Iterating over existing faqBot json and changing priorities based on clicks received in priority dataframe
            #if there are no clicks on any questions then default value for them will be set to 20 so while sorting it is at last
            for l in range(len(faqBot)):
                if faqBot[l]["catId"] in priority_rank:
                    faqBot[l]["priority"]=priority_rank[faqBot[l]["catId"]]
                else:
                    faqBot[l]["priority"]=20

                for q in range(len(faqBot[l]["recommendedQues"])):

                    if faqBot[l]["recommendedQues"][q]["quesID"] in priority_rank:
                        faqBot[l]["recommendedQues"][q]["priority"]=priority_rank[faqBot[l]["recommendedQues"][q]["quesID"]]
                    else:
                        faqBot[l]["recommendedQues"][q]["priority"]=20

                    for q2 in range(len(faqBot[l]["recommendedQues"][q]["recommendedQues"])):
                        if faqBot[l]["recommendedQues"][q]["recommendedQues"][q2]["quesID"] in priority_rank:
                            faqBot[l]["recommendedQues"][q]["recommendedQues"][q2]["priority"]=priority_rank[faqBot[l]["recommendedQues"][q]["recommendedQues"][q2]["quesID"]]
                        else:
                            faqBot[l]["recommendedQues"][q]["recommendedQues"][q2]["priority"]=20
                            
        elif p_data==0:          

            for l in range(len(faqBot)):
                if faqBot[l]["catId"] in priority_rank:
                    faqBot[l]["priority"]=priority_rank[faqBot[l]["catId"]]
                else:
                    faqBot[l]["priority"]=1

                for q in range(len(faqBot[l]["recommendedQues"])):

                    if faqBot[l]["recommendedQues"][q]["quesID"] in priority_rank:
                        faqBot[l]["recommendedQues"][q]["priority"]=priority_rank[faqBot[l]["recommendedQues"][q]["quesID"]]
                    else:
                        faqBot[l]["recommendedQues"][q]["priority"]=1

                    for q2 in range(len(faqBot[l]["recommendedQues"][q]["recommendedQues"])):
                        if faqBot[l]["recommendedQues"][q]["recommendedQues"][q2]["quesID"] in priority_rank:
                            faqBot[l]["recommendedQues"][q]["recommendedQues"][q2]["priority"]=priority_rank[faqBot[l]["recommendedQues"][q]["recommendedQues"][q2]["quesID"]]
                        else:
                            faqBot[l]["recommendedQues"][q]["recommendedQues"][q2]["priority"]=1          
        config.append(faqBot)          # config empty list will capture recommendation maps of Issue and information
            ##################################################
            
        def sort_by_priority(item):        # Method for sorting of recommendation map 
            if 'recommendedQues' in item:
                item['recommendedQues'] = sorted(item['recommendedQues'], key=lambda x: x['priority'])
                for sub_item in item['recommendedQues']:
                    sort_by_priority(sub_item)
        all_faqBot={}                      # empty dict to capture sorted recommendation maps
        
        data_sorted = sorted(config[0], key=lambda x: x['priority'])
        
        for item in data_sorted:
            sort_by_priority(item)
        
        all_faqBot["recommendationMap"]=data_sorted
        ##################################################
        
        all_json_dump={}
        all_json_dump.update(self.cat_map)
        all_json_dump.update(all_faqBot)
        all_json_dump.update(self.que_map)      
        data_output = {"config": all_json_dump}
        print()
         ##################################################
        
        def seq_priority(hirarchy):
            count_cat=1
            for l in range(len(hirarchy["config"]["recommendationMap"])):

                hirarchy["config"]["recommendationMap"][l]["priority"]=count_cat
                count_L1=1


                for q in range(len(hirarchy["config"]["recommendationMap"][l]["recommendedQues"])):

                    hirarchy["config"]["recommendationMap"][l]["recommendedQues"][q]["priority"]=count_L1

                    count_L2=1
                    for q2 in range(len(hirarchy["config"]["recommendationMap"][l]["recommendedQues"][q]["recommendedQues"])):
                        hirarchy["config"]["recommendationMap"][l]["recommendedQues"][q]["recommendedQues"][q2]["priority"]=count_L2
                        count_L2+=1    
                    count_L1+=1
                count_cat+=1
#             return hirarchy
        
        seq_priority(data_output)
#         print(data_output)

        return data_output
    


In [205]:
a=api("EN", "IN")

In [142]:
a.generate_priorit_recommend(df1)

AttributeError: 'int' object has no attribute 'startswith'

In [143]:

df1=pd.read_excel(r"C:\Users\Palash Ashok Bhosale\Jupy\Projects\FAQ - Copy\palash\priority_sort\priority_dff.xlsx")

In [73]:
df1.set_index("qid", inplace=True)

In [ ]:
#importing Libraries
import pandas as pd
import numpy as np
import json

# importing path of excel and Json files 

path_excel=r"C:\Users\Palash Ashok Bhosale\Jupy\Projects\cbrelease\faq-assistant\data\faq.xlsx"
# path_priority=r"data/temp_PRIORITY.xlsx"


# Creating class api
class api():
    
    # while calling class API, we must pass lang=Language("EN", "HI", etc) and 
    # p_cat=parent category("IN": information, "IS": issue)

    # while calling class API, we must pass lang=Language("EN", "HI", etc) and 
    # p_cat=parent category("IN": information, "IS": issue)

    def __init__(self, lang, p_cat):
        self.lang = lang
        self.p_cat = p_cat
        self.load_paths()
        self.load_dataframes()
        self.load_all_maps()
        ####################################################
        
    def load_paths(self):
        # Define file paths based on language and parent category
        base_path = r"data"
        self.path_recommend = f"{base_path}/{self.p_cat}_recommendation_{self.lang}.json"
        self.path_cat_map = f"{base_path}/{self.p_cat}_cat_map_{self.lang}.json"
        self.path_que_map = f"{base_path}/{self.p_cat}_qa_{self.lang}.json"
        self.path_excel = r"data/faq.xlsx"
#         self.path_priority = r"data/temp_PRIORITY.xlsx"
        ####################################################
        
    def load_dataframes(self):
        # as all the data is in on excel file we need to fetch specific sheet
        sheet_name=f"{self.p_cat}{self.lang}"
        self.df = pd.read_excel(self.path_excel, sheet_name=sheet_name)
        ####################################################
        
    def load_all_maps(self):
        with open(self.path_recommend, "r", encoding="utf-8") as rec_file:
            rec_data = json.load(rec_file)
            self.recommend1 = rec_data["faqBot"]

        with open(self.path_cat_map, "r", encoding="utf-8") as cat_file:
            cat_data = json.load(cat_file)
            self.cat_map = cat_data

        with open(self.path_que_map, "r", encoding="utf-8") as que_file:
            que_data = json.load(que_file)
            self.que_map = que_data
        ####################################################
        
    def generate_priorit_recommend(self, p_data=0): #method for giving priority for existing recommendation_map
        global data

        
        config=[]
        p=1
        data=self.df
        faqBot=self.recommend1     
        lang=self.lang
        ####################################################
        
        dict_container = {}                # Create a dictionary to hold other dictionaries
        dict_container[f"cat{p}_clicks"] = {}
        for i in range(1,len(data["Category"].unique())+1):#for loop to auto generate dictionaary for no. of categories 
            dict_name = f"cat{i}_L1_clicks" # Create the dynamic dictionary name
            dict_container[dict_name] = {}  # Create an empty dictionary and store it in the container
            dict_name = f"cat{i}_L2_clicks"  
            dict_container[dict_name] = {}  

        priority_rank={} #initiating dict to store ids and respective priority rank
        ##################################################

        #Iterating over existing recommendation map and setting priority based on priority rank dictionary 
        if type(p_data)!=int:# checking if priority dataframe is passed otherwise p_data is taken as 0 by default
            num=1 # num variable is used for incrementation
            
            for index,row in p_data.iterrows():#for loop for iterating over priority dataframe
                if index.startswith(f"{self.p_cat}{self.lang}C") and 0<len(index)<=8:
                    dict_container[f"cat{p}_clicks"][index]=row["clicks"]# creating cat clicks dict in parent dict 

                for i in range(1,len(data["Category"])):# creating L1 and L2 clicks dict in parent dict
                    if index.startswith(f"{self.p_cat}{self.lang}C10{i}") and 8<len(index)==14:
                        dict_container[f"cat{i}_L1_clicks"][index]=row['clicks']

                    elif index.startswith(f"{self.p_cat}{self.lang}C10{i}") and 14<len(index)==20:
                        dict_container[f"cat{i}_L2_clicks"][index]=row['clicks']
            num+=1
            ####################################################
            
            for k,v in dict_container.items():# for loop for reverse sorting of all dicts in parent dict
                dict_container[k]={k:v for k,v in (sorted(dict_container[k].items(), key=lambda x:x[1],reverse=True))}          
            count=1 #count variable is initiated to set priority rank for sorted ids from 1 to length of that dict

            for key,value in dict_container.items():# for loop for iterating over each dictionary in parent dict

                for k,v in dict_container[key].items():# for loop for iterating over key i.e dicts in parent dict 
                    priority_rank[k]=count                             # and value i.e dict having ids and clicks
                    count+=1
                count=1 
            count=1
           #################################################### 

            # grouping l1 level questions and l2 level questions based on ids
            # L1 and L2 ids are captured by iterating over existing recommendation map
            l=[]
            for cc in range(len(data["Category"].unique())):
                for i in range (len(faqBot[cc]["recommendedQues"])):
                    L1_id=faqBot[cc]["recommendedQues"][i]["quesID"] 
                    for j in range (len(faqBot[cc]["recommendedQues"][i]["recommendedQues"])):
                        L2_id=faqBot[cc]["recommendedQues"][i]["recommendedQues"][j]["quesID"]
                        l.extend([[L1_id, L2_id]])
            ####################################################            
            li={}# creating dictionary for capturing L1 ids of particular category and
                 #L2 ids of each different L1 qids as key and value is clicks on that ids 
                 #Fetching clicks of that id from priority dataframe passed in this function
            a=0
            for i in l:
                if l[a][0] not in li.keys():
                    try:
                        li[l[a][0]]= [[p_data.loc[l[a][1]][0], l[a][1]]] #if id is not in dict "li" then new key will be created
                    except:
                        print("no id found in telemetry for:" + str(i))

                else:
                    try:
                        li[l[a][0]].extend([[p_data.loc[l[a][1]][0], l[a][1]]]) #if id is in dict "li" then value will be added
                    except:
                        print("no id found in telemetry for:" + str(i))
                a+=1
            a=0
            ####################################################
            
            for k,v in li.items():
                num=1
                lll=sorted(v, reverse=True)

                for i in range(len(lll)):
                    priority_rank[lll[i][1]]=num  # populating priority based on number of clicks
                    num+=1
                num=1
            ####################################################
            
            #Iterating over existing faqBot json and changing priorities based on clicks received in priority dataframe
            #if there are no clicks on any questions then default value for them will be set to 20 so while sorting it is at last
            for l in range(len(faqBot)):
                if faqBot[l]["catId"] in priority_rank:
                    faqBot[l]["priority"]=priority_rank[faqBot[l]["catId"]]
                else:
                    faqBot[l]["priority"]=20

                for q in range(len(faqBot[l]["recommendedQues"])):

                    if faqBot[l]["recommendedQues"][q]["quesID"] in priority_rank:
                        faqBot[l]["recommendedQues"][q]["priority"]=priority_rank[faqBot[l]["recommendedQues"][q]["quesID"]]
                    else:
                        faqBot[l]["recommendedQues"][q]["priority"]=20

                    for q2 in range(len(faqBot[l]["recommendedQues"][q]["recommendedQues"])):
                        if faqBot[l]["recommendedQues"][q]["recommendedQues"][q2]["quesID"] in priority_rank:
                            faqBot[l]["recommendedQues"][q]["recommendedQues"][q2]["priority"]=priority_rank[faqBot[l]["recommendedQues"][q]["recommendedQues"][q2]["quesID"]]
                        else:
                            faqBot[l]["recommendedQues"][q]["recommendedQues"][q2]["priority"]=20
                            
        elif p_data==0:          

            for l in range(len(faqBot)):
                if faqBot[l]["catId"] in priority_rank:
                    faqBot[l]["priority"]=priority_rank[faqBot[l]["catId"]]
                else:
                    faqBot[l]["priority"]=1

                for q in range(len(faqBot[l]["recommendedQues"])):

                    if faqBot[l]["recommendedQues"][q]["quesID"] in priority_rank:
                        faqBot[l]["recommendedQues"][q]["priority"]=priority_rank[faqBot[l]["recommendedQues"][q]["quesID"]]
                    else:
                        faqBot[l]["recommendedQues"][q]["priority"]=1

                    for q2 in range(len(faqBot[l]["recommendedQues"][q]["recommendedQues"])):
                        if faqBot[l]["recommendedQues"][q]["recommendedQues"][q2]["quesID"] in priority_rank:
                            faqBot[l]["recommendedQues"][q]["recommendedQues"][q2]["priority"]=priority_rank[faqBot[l]["recommendedQues"][q]["recommendedQues"][q2]["quesID"]]
                        else:
                            faqBot[l]["recommendedQues"][q]["recommendedQues"][q2]["priority"]=1          
        config.append(faqBot)          # config empty list will capture recommendation maps of Issue and information
            ##################################################
            
        def sort_by_priority(item):        # Method for sorting of recommendation map 
            if 'recommendedQues' in item:
                item['recommendedQues'] = sorted(item['recommendedQues'], key=lambda x: x['priority'])
                for sub_item in item['recommendedQues']:
                    sort_by_priority(sub_item)
        all_faqBot={}                      # empty dict to capture sorted recommendation maps
        
        data_sorted = sorted(config[0], key=lambda x: x['priority'])
        
        for item in data_sorted:
            sort_by_priority(item)
        
        all_faqBot["recommendationMap"]=data_sorted
        ##################################################
        
        all_json_dump={}
        all_json_dump.update(self.cat_map)
        all_json_dump.update(all_faqBot)
        all_json_dump.update(self.que_map)      
        data_output = {"config": all_json_dump}

         ##################################################
        
        def seq_priority(hirarchy):
            count_cat=1
            for l in range(len(hirarchy["config"]["recommendationMap"])):

                hirarchy["config"]["recommendationMap"][l]["priority"]=count_cat
                count_L1=1


                for q in range(len(hirarchy["config"]["recommendationMap"][l]["recommendedQues"])):

                    hirarchy["config"]["recommendationMap"][l]["recommendedQues"][q]["priority"]=count_L1

                    count_L2=1
                    for q2 in range(len(hirarchy["config"]["recommendationMap"][l]["recommendedQues"][q]["recommendedQues"])):
                        hirarchy["config"]["recommendationMap"][l]["recommendedQues"][q]["recommendedQues"][q2]["priority"]=count_L2
                        count_L2+=1    
                    count_L1+=1
                count_cat+=1
#             return hirarchy
        
        seq_priority(data_output)


        return data_output
    
